In [ ]:
import pandas as pd
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.preprocessing import StandardScaler, LabelEncoder, OneHotEncoder
from scikeras.wrappers import KerasClassifier
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Input

In [11]:
data=pd.read_csv('Churn_Modelling.csv')
data = data.drop(['RowNumber', 'CustomerId', 'Surname'], axis=1)

label_encoder_gender = LabelEncoder()
data['Gender'] = label_encoder_gender.fit_transform(data['Gender'])

onehot_encoder_geo = OneHotEncoder(handle_unknown='ignore')
geo_encoded = onehot_encoder_geo.fit_transform(data[['Geography']]).toarray()
geo_encoded_df = pd.DataFrame(geo_encoded, columns=onehot_encoder_geo.get_feature_names_out(['Geography']))

data = pd.concat([data.drop('Geography', axis=1), geo_encoded_df], axis=1)

X = data.drop('Exited', axis=1)
y = data['Exited']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

In [ ]:
## Define a function to create the model and try different parameters(KerasClassifier)

def create_model(neurons=32,layers=1):
    model=Sequential([Input(shape=(X_train.shape[1],))])
    model.add(Dense(neurons,activation='relu'))

    for _ in range(layers-1):
        model.add(Dense(neurons,activation='relu'))

    model.add(Dense(1,activation='sigmoid'))
    model.compile(optimizer='adam',loss="binary_crossentropy",metrics=['accuracy'])

    return model

In [13]:
## Create a Keras classifier
model=KerasClassifier(layers=1,neurons=32,model=create_model,verbose=1)

In [14]:
# Define the grid search parameters
param_grid = {
    'neurons': [16, 32, 64, 128],
    'layers': [1, 2],
    'epochs': [50, 100]
}

In [15]:
# Perform grid search
grid = GridSearchCV(estimator=model, param_grid=param_grid, n_jobs=-1, cv=3,verbose=1)
grid_result = grid.fit(X_train, y_train)

# Print the best parameters
print("Best: %f using %s" % (grid_result.best_score_, grid_result.best_params_))

Fitting 3 folds for each of 16 candidates, totalling 48 fits
Epoch 1/100
250/250 ━━━━━━━━━━━━━━━━━━━━ 1s 908us/step - accuracy: 0.6060 - loss: 0.6826
Epoch 2/100
250/250 ━━━━━━━━━━━━━━━━━━━━ 0s 807us/step - accuracy: 0.8011 - loss: 0.4630
Epoch 3/100
250/250 ━━━━━━━━━━━━━━━━━━━━ 0s 812us/step - accuracy: 0.8111 - loss: 0.4354
Epoch 4/100
250/250 ━━━━━━━━━━━━━━━━━━━━ 0s 802us/step - accuracy: 0.8184 - loss: 0.4186
Epoch 5/100
250/250 ━━━━━━━━━━━━━━━━━━━━ 0s 839us/step - accuracy: 0.8279 - loss: 0.4040
Epoch 6/100
250/250 ━━━━━━━━━━━━━━━━━━━━ 0s 843us/step - accuracy: 0.8365 - loss: 0.3907
Epoch 7/100
250/250 ━━━━━━━━━━━━━━━━━━━━ 0s 785us/step - accuracy: 0.8426 - loss: 0.3799
Epoch 8/100
250/250 ━━━━━━━━━━━━━━━━━━━━ 0s 803us/step - accuracy: 0.8456 - loss: 0.3709
Epoch 9/100
250/250 ━━━━━━━━━━━━━━━━━━━━ 0s 811us/step - accuracy: 0.8496 - loss: 0.3643
Epoch 10/100
250/250 ━━━━━━━━━━━━━━━━━━━━ 0s 805us/step - accuracy: 0.8533 - loss: 0.3591
Epoch 11/100
250/250 ━━━━━━━━━━━━━━━━━━━━ 0s 800